**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

### Install HuggingFace transformers & sacremoses dependency

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [ ]:
!pip install ../input/sacremoses/sacremoses-master/
!pip install ../input/transformers/transformers-master/

### Required Imports

I've added imports that will be used in training too

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
DATA_DIR = '../input/google-quest-challenge'

In [ ]:
!ls ../input

In [ ]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

In [ ]:
target_columns = sub.columns.values[1:].tolist()
target_columns

### Define dataset

In [ ]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

In [ ]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

In [ ]:
import torch
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
from sklearn.utils import shuffle
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

MAX_LEN = 512
#MAX_Q_LEN = 250
#MAX_A_LEN = 259
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.tokenizer = BertTokenizer.from_pretrained('../input/d/datasets/abhishek/bert-base-uncased/')
        
        self.tokenizer = get_tokenizer('basic_english')

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids, seg_ids = self.get_token_ids(row)
        
        self.vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
        self.vocab.set_default_index(vocab['<unk>'])
        
        if self.labeled:
            labels = self.get_label(row)
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

    def __len__(self):
        return len(self.df)

    def select_tokens(self, tokens, max_num):
        if len(tokens) <= max_num:
            return tokens
        if self.train_mode:
            num_remove = len(tokens) - max_num
            remove_start = random.randint(0, len(tokens)-num_remove-1)
            return tokens[:remove_start] + tokens[remove_start + num_remove:]
        else:
            return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]

    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=30, q_max_len=239, a_max_len=239):
        t = self.tokenizer.tokenize(title)
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)

        t_len = len(t)
        q_len = len(q)
        a_len = len(a)

        if (t_len+q_len+a_len+4) > max_sequence_length:

            if t_max_len > t_len:
                t_new_len = t_len
                a_max_len = a_max_len + floor((t_max_len - t_len)/2)
                q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
            else:
                t_new_len = t_max_len

            if a_max_len > a_len:
                a_new_len = a_len 
                q_new_len = q_max_len + (a_max_len - a_len)
            elif q_max_len > q_len:
                a_new_len = a_max_len + (q_max_len - q_len)
                q_new_len = q_len
            else:
                a_new_len = a_max_len
                q_new_len = q_max_len


            if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]

        return t, q, a
        
    def get_token_ids(self, row):
        t_tokens, q_tokens, a_tokens = self.trim_input(row.question_title, row.question_body, row.answer)

        tokens = ['[CLS]'] + t_tokens + ['[SEP]'] + q_tokens + ['[SEP]'] + a_tokens + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            token_ids += [0] * (MAX_LEN - len(token_ids))
        ids = torch.tensor(token_ids)
        seg_ids = self.get_seg_ids(ids)
        return ids, seg_ids
    
    def get_seg_ids(self, ids):
        seg_ids = torch.zeros_like(ids)
        seg_idx = 0
        first_sep = True
        for i, e in enumerate(ids):
            seg_ids[i] = seg_idx
            if e == SEP_TOKEN_ID:
                if first_sep:
                    first_sep = False
                else:
                    seg_idx = 1
        pad_idx = torch.nonzero(ids == 0)
        seg_ids[pad_idx] = 0

        return seg_ids

    def get_label(self, row):
        print(row[target_columns].values)
        return torch.tensor(row[target_columns].values.astype(np.float32))

    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
    
        if self.labeled:
            labels = torch.stack([x[2] for x in batch])
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')
    ds_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=ds_test.collate_fn, drop_last=False)
    loader.num = len(df)
    
    return loader
        
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')
    df = shuffle(df, random_state=1234)
    #split_index = int(len(df) * (1-val_percent))
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    #print(df_val.head())
    #df_train = df[:split_index]
    #df_val = df[split_index:]

    print(df_train.shape)
    print(df_val.shape)

    ds_train = QuestDataset(df_train)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=ds_train.collate_fn, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, collate_fn=ds_val.collate_fn, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader

def test_train_loader():
    loader, _ = get_train_val_loaders(4, 4, 1)
    for ids, seg_ids, labels in loader:
        print("ids: ",ids)
        print("seg_ids: ",seg_ids.numpy())
        print("labels: ",labels)
        break
def test_test_loader():
    loader = get_test_loader(4)
    for ids, seg_ids in loader:
        print("ids: ",ids)
        print("seg_ids: ",seg_ids)
        break

In [ ]:
pwd

In [ ]:
ls ../input/d/datasets/abhishek/bert-base-uncased

## Build Model

In [ ]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self,ids, n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        #self.bert_model = BertModel.from_pretrained('../input/d/datasets/abhishek/bert-base-uncased/')
        
        #ntokens = len(vocab)  # size of vocabulary
        ntokens = len(ids)  # size of vocabulary
        emsize = 200  # embedding dimension
        d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
        nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
        nhead = 2  # number of heads in nn.MultiheadAttention
        dropout = 0.2  # dropout probability
        
        self.transformer_model=TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout)
        
        self.fc = nn.Linear(768, n_classes)
        
    def forward(self, ids, seg_ids):
        attention_mask = (ids > 0)
        #layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        
        out =  self.transformer_model(ids, seg_ids)
        
        #print(layers[-1][0].size())
        #print(pool_out.size())

        #out = F.dropout(layers[-1][:, 0, :], p=0.2, training=self.training)
        #out =  F.dropout(pool_out, p=0.2, training=self.training)
        
        logit = self.fc(out)
        return logit
    
def test_model():
    x = torch.tensor([[1,2,3,4,5, 0, 0], [1,2,3,4,5, 0, 0]])
    seg_ids = torch.tensor([[0,0,0,0,0, 0, 0], [0,0,0,0,0, 0, 0]])
    model = QuestModel(x)

    y = model(x, seg_ids)
    print(y)

In [ ]:
!ls ../input

In [ ]:
def create_model(ids):
    model = QuestModel(ids)
    #model.load_state_dict(torch.load(model_file))
    model = model.cuda()
    #model = DataParallel(model)
    return model

def create_models(ids):
    models = []
    for i in range(5):
        model = create_model(ids)
        model.eval()
        models.append(model)
    return models

In [ ]:
from tqdm import tqdm
import torch
def predict(models, test_loader):
    all_scores = []
    with torch.no_grad():
        for ids, seg_ids in tqdm(test_loader, total=test_loader.num // test_loader.batch_size):
            ids, seg_ids = ids.cuda(), seg_ids.cuda()
            scores = []
            for model in models:
                outputs = torch.sigmoid(model(ids, seg_ids)).cpu()
                scores.append(outputs)
            all_scores.append(torch.mean(torch.stack(scores), 0))

    all_scores = torch.cat(all_scores, 0).numpy()
    
    return all_scores

In [ ]:
test_loader = get_test_loader(batch_size=32)

In [ ]:
models = create_models(test_loader)

In [ ]:
preds = predict(models, test_loader)

In [ ]:
preds[:1]

In [ ]:
preds

### Generate Submission

In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

In [ ]:
sub[target_columns] = preds

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()